# Removing Dateless items

In [79]:
import pandas as pd
import datetime
import itertools


In [2]:
dates =pd.read_table("dates.txt", header = None)
dates.columns =['id1', 'date']

In [3]:
raw = pd.read_table("data.txt", header = None)

In [4]:
dataset =set(raw.iloc[:,1].tolist()+raw.iloc[:,0].tolist())

In [5]:
dateset =set(dates.iloc[:,0])

In [6]:
uniq = list(dataset & dateset)

In [7]:
isuniq = raw.isin(uniq)

In [8]:
data = raw[isuniq.iloc[:,1] & isuniq.iloc[:,0]]
data.columns =['id1', 'id2']

In [9]:
data.shape

(347268, 2)

In [10]:
raw.shape

(421578, 2)

In [11]:
data = data.merge(dates, on="id1")

In [12]:
data['date'] = pd.to_datetime(data['date'])
data  = data.iloc[data['date'].argsort(),:]

In [18]:
data = data.reset_index()
del data['index']

In [22]:
data.head()

,id1,id2,date
0,9203210,9801208,1992-03-15
1,9203210,9801209,1992-03-15
2,9203222,9908362,1992-03-27
3,9204229,9908362,1992-04-22
4,9205220,9506358,1992-05-13


# Sample Creation

In [23]:
trainingsize= 1001
for i in xrange(data.shape[0]):
    if len(set(data['id1'][0:i]))==trainingsize:
        print i-1
        break
        
validationsize = 50
for i in xrange(data.shape[0]):
    if len(set(data['id1'][2745:i]))==validationsize:
        print i-1
        break



2745
2910


In [275]:
sample = data.iloc[0:2745]


In [276]:
sampleset =set(sample['id1'].tolist()+sample['id2'].tolist())
dictionary =dict(zip(sampleset,range(1,len(sampleset)+1)))
d =pd.DataFrame(dictionary.items())
d.columns = ['id','node']
sample=sample.merge(d, left_on ='id1', right_on = 'id')
sample=sample.merge(d, left_on ='id2', right_on = 'id')
del sample['id_x']
del sample['id_y']

In [277]:
sample.head()

,id1,id2,date,node_x,node_y
0,9203210,9801208,1992-03-15,638,631
1,9303219,9801208,1993-03-04,1073,631
2,9310203,9801208,1993-10-01,654,631
3,9203210,9801209,1992-03-15,638,632
4,9303219,9801209,1993-03-04,1073,632


In [278]:
d.shape

(1461, 2)

In [269]:
validation = data.iloc[2745:2910,:]

In [270]:
validset =set(validation['id1'].tolist()+validation['id2'].tolist())
new = validset-sampleset
dictionary2 =dict(zip(new,range(len(sampleset)+1,len(sampleset)+1+len(new))))

In [271]:
dictionary.update(dictionary2)
d2 =pd.DataFrame(dictionary.items())
d2.columns = ['id','node']
validation=validation.merge(d2, left_on ='id1', right_on = 'id')
validation=validation.merge(d2, left_on ='id2', right_on = 'id')
del validation['id_x']
del validation['id_y']

In [280]:
d2.shape

(1524, 2)

In [272]:
validation.head()

,id1,id2,date,node_x,node_y
0,9312285,9204214,1993-12-15,1464,916
1,9312287,9207213,1993-12-15,1465,1420
2,9312287,9207209,1993-12-15,1465,1417
3,9312331,9207209,1993-12-23,1497,1417
4,9312287,9302316,1993-12-15,1465,842


In [273]:
sample.to_csv('training.csv',index  = False)
validation.to_csv('validation.csv',index  = False)